In [1]:
import os
import asyncio
import yaml
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_core.models import ChatCompletionClient, ModelInfo
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination, TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, UnstructuredWordDocumentLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
AZURE = "C:/Master IA/TFM_BA_GPT/src/model_config_azure.yaml"
OLLAMA = "C:/Master IA/TFM_BA_GPT/src/model_config_ollama.yaml"
LMSTUDIO = "C:/Master IA/TFM_BA_GPT/src/model_config_lmstudio.yaml"

# Load model configuration and create the model client.
with open(OLLAMA, "r") as f:
    model_config = yaml.safe_load(f)
    
model_client = ChatCompletionClient.load_component(model_config)

In [3]:
### --- CONFIG PATH--- ###
CHROMA_DIR = "../chromadb_storage"
COLLECTION_NAME = "rag_collection"

In [4]:
### --- RAG-ENABLED AUTO-GEN SETUP WITH CHROMADB --- ###

vector_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name=COLLECTION_NAME,
        persistence_path=CHROMA_DIR,
        k=3,
        score_threshold=0.4,
    )
)

In [14]:
collection = vector_memory._collection
collection.peek(5)

{'ids': ['abd15623-b560-41b9-a8ae-a2eacf3ca02e',
  'c16db792-265c-41b7-a065-c698bf63d6fc',
  '14ce947b-4c1b-4606-bcc5-86eeb29d0d17',
  'f522bf29-b8f3-4a23-ac25-7bb08ac6ef49',
  'b7f87563-0938-448c-a1ce-a849fe1a2b77'],
 'embeddings': array([[-0.02579624,  0.03621831,  0.0216359 , ...,  0.05376932,
         -0.04428232,  0.04766113],
        [ 0.05751115,  0.0712025 ,  0.03817087, ...,  0.06237269,
         -0.08207577,  0.06998023],
        [-0.00181749,  0.01720334, -0.02061422, ...,  0.04590431,
          0.00495566,  0.10591003],
        [-0.01633696,  0.04933928, -0.03204421, ...,  0.02683388,
         -0.06981689,  0.07045751],
        [-0.03346531,  0.05374684, -0.02878887, ...,  0.10393491,
         -0.01113013,  0.01846812]], shape=(5, 384)),
 'documents': ["11 Effective Questions that every good BA should ask to gather the answer when starting a \nproject \nIt is a list of obvious questions that every good BA should ask to gather the answer when \nstarting a project. \n \nTell 

Spec virifier and document writter agents

In [5]:
def read_txt_file(file_path: str) -> str:
    """
    Reads the content of a .txt file and returns it as a string.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"
    
def read_pdf_file(file_path: str) -> str:
    """
    Reads the content of a .pdf file and returns it as a string.
    """
    try:
        loader = PyMuPDFLoader(file_path)
        documents = loader.load()
        return documents[0].page_content if documents else ""
    except Exception as e:
        return f"Error reading file: {e}"

In [6]:
funct_spec_checker_agent = AssistantAgent(
    name="func_spec_checker",
    description="An agent that checks if the functional specification document is complete.",
    model_client=model_client,
    memory=[vector_memory],
    system_message="""
    You are a Document Checker Business Analyst.
    You are responsible for checking the functional specification document is complete and cover all the subjects below:

    1. Executive Summary 
    1.1. Overview
    1.2. Business Benefits
    2. Scope 
    2.1. In Scope
    2.2. Out of Scope
    2.3. Systems & Platforms
    3. Requirements 
    3.1. Functional Requirements
    3.2. Non-Functional Requirements
    4. Functional Solution 
    4.1. Overview
    4.2. Impact Assessment
    4.3. Assumptions
    4.4. Dependencies
    4.5. Entities
    5. Business Processes 
    5.1 Process Description AS IS
    5.2. Process Description TO BE
    6. Stories 
    6.1 Stories List
    6.2 User Stories
    7. Risk Assessment 
    7.1 Risk Matrix

    If the document is complete, please say: "The functional specification document is complete" and then "TERMINATE"
    If the document is not complete, please say: "The functional specification document is not complete" 
    and provide a list of the missing subjects and then "TERMINATE"
    """,
)

doc_writter_agent = AssistantAgent(
    name="doc_writter",
    description="An agent that writes the functional specification document.",
    model_client=model_client,
    tools=[read_pdf_file],
    memory=[vector_memory],
    system_message="""
    You are a Document Writter Business Analyst, this agent should be the first to engage when given a new task.
    You are responsible for writing the functional specification document in the functional specification document template format.
    """,
)

In [7]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [8]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure to start with the doc_writter.
Only select one agent.
"""

In [11]:
team = SelectorGroupChat(
    [funct_spec_checker_agent, doc_writter_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [ ]:
task = "Read the pdf file in file_path = 'C:/Master IA/TFM_BA_GPT/data/Tema.pdf' and write the functional specification document in the functional specification document template format then verify if the document is complete."

In [13]:
await Console(team.run_stream(task=task))

---------- user ----------
Read the text file in file_path = 'C:/Master IA/TFM_BA_GPT/data/Tema.pdf' and write the functional specification document in the functional specification document template format then verify if the document is complete.
---------- doc_writter ----------
[MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDocument Status IN PROGRESS ONHOLD COMPLETED \nAuthor(s) \n[BA name] \nSponsor \n[Sponsor name] \nRelated Area \n[area related to the sponsor] \n \n1. Document Information \n \n1.1. Document Approvers & Reviewers \nName \nRole \nApprover / \nReviewer \nApproval / \nReview Date \nApproved / \nReview Version \nHead of UAT \nHead of UAT \nApprover \n  \n  \nHead of Business \nChange', mime_type='MemoryMimeType.TEXT', metadata={'chunk_index': 0, 'mime_type': 'MemoryMimeType.TEXT', 'source': 'Functional Requirements Specification Template.doc.pdf', 'score': 0.

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Read the text file in file_path = 'C:/Master IA/TFM_BA_GPT/data/Tema.pdf' and write the functional specification document in the functional specification document template format then verify if the document is complete.", type='TextMessage'), MemoryQueryEvent(source='doc_writter', models_usage=None, metadata={}, content=[MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDocument Status IN PROGRESS ONHOLD COMPLETED \nAuthor(s) \n[BA name] \nSponsor \n[Sponsor name] \nRelated Area \n[area related to the sponsor] \n \n1. Document Information \n \n1.1. Document Approvers & Reviewers \nName \nRole \nApprover / \nReviewer \nApproval / \nReview Date \nApproved / \nReview Version \nHead of UAT \nHead of UAT \nApprover \n  \n  \nHead of Business \nChange', mime_type='MemoryMimeType.TEXT', metadata={'ch